In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer


In [2]:
# Load pre-trained models
scene_recognition_model = ResNet50(weights='imagenet')


In [3]:
# !pip install --upgrade keras

In [4]:
import keras
action_recognition_model = keras.applications.InceptionV3(include_top=True,weights="imagenet",input_tensor=None,input_shape=None,pooling=None,classes=1000,classifier_activation="softmax",)


In [5]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')


C:\Users\kruti\anaconda3\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [6]:
# !pip install transformers

In [7]:
# tokenizer = AutoTokenizer.from_pretrained('google/vit-base-patch16-224-in21k')
# from transformers import AutoModelForVisionClassification

# tokenizer = AutoModelForVisionClassification.from_pretrained('google/vit-base-patch16-224-in21k')



In [8]:
caption_generation_model = VisionEncoderDecoderModel.from_pretrained('nlpconnect/vit-gpt2-image-captioning')


Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.11.crossattention.bias', 'decoder.transformer.h.5.crossattention.masked_bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.1.attn.masked_bias', 'decoder.transformer.h.11.attn.bias', 'decoder.transformer.h.5.crossattention.bias', 'decoder.transformer.h.8.crossattention.masked_bias', 'decoder.transformer.h.2.attn.bias', 'decoder.transformer.h.8.crossattention.bias', 'decoder.transformer.h.0.crossattention.masked_bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.11.attn.masked_bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.9.attn.masked_bias', 'decoder.transformer.h.0.crossattention.bias', 'decoder.transformer.h.10.attn.bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.3.crossattent

In [9]:
def extract_frames(video_path, frame_rate, batch_size=32):
    vidcap = cv2.VideoCapture(video_path)
    frames = []
    success, image = vidcap.read()
    count = 0
    while success:
        if count % frame_rate == 0:
            frames.append(image)
        if len(frames) == batch_size:
            yield frames
            frames = []
        success, image = vidcap.read()
        count += 1
    if frames:
        yield frames

In [10]:
with open('D:/m.tech/data/Data Set/942d3a0ac09ec9e5eb3a-238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt', 'r') as f:
    class_labels = [line.strip() for line in f.readlines()]
def recognize_scene(frame):
    # Resize the frame to the expected input shape of the model
    frame = cv2.resize(frame, (224, 224))

    x = preprocess_input(np.expand_dims(frame.copy(), axis=0))
    preds = scene_recognition_model.predict(x)
    top_class_idx = np.argmax(preds)
    top_class_label = class_labels[top_class_idx]

    return top_class_label

In [11]:
def recognize_action(frames):
    # Resize each frame individually
    resized_frames = [cv2.resize(frame, (299, 299)) for frame in frames]

    inputs = np.stack([preprocess_input(frame.copy()) for frame in resized_frames])
    preds = action_recognition_model.predict(inputs)

    # Assuming the action_recognition_model is the InceptionV3 model
    # You'll need to replace this line with the appropriate way to get the class label
    top_class_idx = np.argmax(preds)
    action_label = class_labels[top_class_idx]  # Replace with the appropriate class labels list

    return action_label

In [12]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlpconnect/vit-gpt2-image-captioning')

In [13]:
caption_generation_model = VisionEncoderDecoderModel.from_pretrained('nlpconnect/vit-gpt2-image-captioning')
tokenizer = AutoTokenizer.from_pretrained('nlpconnect/vit-gpt2-image-captioning')

Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.11.crossattention.bias', 'decoder.transformer.h.5.crossattention.masked_bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.1.attn.masked_bias', 'decoder.transformer.h.11.attn.bias', 'decoder.transformer.h.5.crossattention.bias', 'decoder.transformer.h.8.crossattention.masked_bias', 'decoder.transformer.h.2.attn.bias', 'decoder.transformer.h.8.crossattention.bias', 'decoder.transformer.h.0.crossattention.masked_bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.11.attn.masked_bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.9.attn.masked_bias', 'decoder.transformer.h.0.crossattention.bias', 'decoder.transformer.h.10.attn.bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.3.crossattent

In [14]:
def generate_caption(frames, scene_labels, action_label):
    pixel_values = feature_extractor(images=frames, return_tensors='pt').pixel_values
    output_ids = caption_generation_model.generate(pixel_values, max_length=50, num_beams=4, early_stopping=True)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return f"Scene: {scene_labels} | Action: {action_label} | Caption: {caption}"

In [15]:
def process_video(video_path, frame_rate=30, batch_size=32):
    frames_generator = extract_frames(video_path, frame_rate, batch_size)
    captions = []
    for frames in frames_generator:
        scene_labels = [recognize_scene(frame) for frame in frames]
        action_label = recognize_action(frames)
        caption = generate_caption(frames, scene_labels, action_label)
        captions.append(caption)
    return '\n'.join(captions)

In [17]:
video_path = 'D:/m.tech/data/Data Set/MSRVTT/videos/all/video547.mp4'
caption = process_video(video_path)
print(caption)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
Scene: ["703: 'park bench',", "837: 'sunglasses, dark glasses, shades',", "837: 'sunglasses, dark glasses, shades',", "936: 'head cabbage',", "796: 'ski mask',", "744: 'projectile, missile',", "416: 'balance beam, beam',", "657: 'missile',", "416: 'balance beam, beam',", "416: 'balance beam, beam',", "150: 'sea lion',", "399: 'abaya',", "389: 'barracouta, snoek',"] | Action: 916: 'web site, website, internet site, site', | Caption: a man riding on the b

In [17]:
video_path = 'D:/m.tech/data/Data Set/MSRVTT/videos/all/video1017.mp4'
caption = process_video(video_path)
print(caption)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Scene: ["851: 'television, television system',", "851: 'television, television system',", "851: 'television, television system',", "851: 'television, television system',", "851: 'television, television system',", "851: 'television, television system',", "851: 'television, television system',", "851: 'television, television system',", "851: 'television, television system',", "851: 'television, television system',", "851: 'television, television system',"

In [22]:
video_path = "D:/vid/VID20220120171416.mp4"
caption = process_video(video_path)
print(caption)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 